Computation of system segregation according to the formula in 
Chan, M. Y., Park, D. C., Savalia, N. K., Petersen, S. E., & Wig, G. S. (2014). Decreased segregation of brain systems across the healthy adult lifespan. Proceedings of the National Academy of Sciences, 111(46), E4997-E5006.

In [68]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import glob
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
import pyreadstat
from pingouin import pairwise_corr
import scipy.stats as chi2_contingency
import scipy.stats as stats
from scipy.spatial import distance
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [69]:
def calc_num_components(a):
    num_pairs = len(a)
    num_components = int(np.ceil(np.sqrt(2 * num_pairs - 1)))
    if num_components * (num_components - 1) == 2 * num_pairs:
        return num_components
    else:
        return 0


def return_dfnc_from_vector(a):
    '''This Function returns a dfnc matrix given a vector
        a : the vector of correlation pairs
    '''
    n = calc_num_components(a)
    out = np.zeros((n, n))

    if not n:
        return out
    
    l_indices = np.tril_indices(n, -1)
    u_indices = np.triu_indices(n, 1)

    out[u_indices] = a
    out[l_indices] = out.T[l_indices]
    
    return out

In [70]:
n_subjects = 48
n_controls = 17
n_controls_1 = 16
n_patients = 31
n_windows = 159
n_corr = 78

In [72]:
worse= [34, 28, 19, 33, 42, 37, 25, 29, 27, 45, 46, 17, 31] # 46
better = [36, 38,35, 40, 41, 43,  39,  32, 18, 30, 26, 24, 23, 22, 21, 20, 44, 47] # 44
worse_dropped = [34, 28, 19, 33, 42, 37, 25, 29, 27, 44, 17, 31]
better_dropped = [36, 38,  35, 40, 41, 43, 39,  32, 18, 30, 26, 24, 23, 22, 21, 20, 45]

In [73]:
# loading state information 
x = loadmat('states_stroke.mat')["a"]
for n in range(0,np.shape(x)[0],1):
    if n==0:
        states = pd.DataFrame(x[n][0])
    else:
        states[n] = pd.DataFrame(x[n][0])
states = states.transpose()

In [74]:
if sys.platform == 'darwin': # OS X
    subject_names = glob.glob('/Users/anna/Documents/LV_Stroke_new/Data/DFNC_LV_no15_0307/WithSM_dfnc_sub*',)
else: # all other operating systems
    print('Please enter the windows system path at this line. Thanks!!!')
subject_names = sorted(subject_names)

In [75]:
# Analysis of system segregation in case of static connectivity
sfnc = loadmat('/Users/anna/Documents/LV_Stroke_new/Data/Mancovan_LV_no15_1607/sfnc_corrs.mat')["static"]
sfnc_1_group_med = pd.DataFrame(return_dfnc_from_vector(pd.DataFrame(sfnc).mean()))

In [14]:
# Computation of static system segregation (please note: negative connectivity paris are set to 0)
seg_11_all = []
for n in range(48):
    dfnc_1_group_med = pd.DataFrame(return_dfnc_from_vector(sfnc[n]))

    dfnc_1_group_med_SM = dfnc_1_group_med.loc[0:7,0:7]*(dfnc_1_group_med.loc[0:7,0:7]>0)
    dfnc_1_group_med_SM = np.array(dfnc_1_group_med_SM)[np.triu_indices(8,1)]
    dfnc_1_group_med_SC = dfnc_1_group_med.loc[8:10,8:10]*(dfnc_1_group_med.loc[8:10,8:10]>0)
    dfnc_1_group_med_SC = np.array(dfnc_1_group_med_SC)[np.triu_indices(3,1)]
    dfnc_1_group_med_CB = dfnc_1_group_med.loc[11:12,11:12]*(dfnc_1_group_med.loc[11:12,11:12]>0)
    dfnc_1_group_med_CB = np.array(dfnc_1_group_med_CB)[np.triu_indices(2,1)]
    within_1 = np.concatenate([dfnc_1_group_med_SM, dfnc_1_group_med_SC, dfnc_1_group_med_CB])
    within_1 = np.sum(within_1)/(len(dfnc_1_group_med_SM) +len(dfnc_1_group_med_SC)+len(dfnc_1_group_med_CB)) 
    dfnc_1_group_med_SM_SC_CB = np.array(dfnc_1_group_med.loc[8:,0:7]*(dfnc_1_group_med.loc[8:,0:7]>0))
    dfnc_1_group_med_SC_CB = np.array(dfnc_1_group_med.loc[8:10,11:]*(dfnc_1_group_med.loc[8:10,11:]>0))
    between_1 = np.concatenate([np.ravel(dfnc_1_group_med_SM_SC_CB),np.ravel(dfnc_1_group_med_SC_CB)])
    between_1 = np.sum(between_1)/(len(np.ravel(dfnc_1_group_med_SM_SC_CB)) +len(np.ravel(dfnc_1_group_med_SC_CB))) 
    seg_11 = (within_1 - between_1) / within_1
    seg_11_all.append(seg_11)

In [15]:
# One-way ANOVA testing for group differences between controls and patient subgroups for static system segregation 
stats.f_oneway(pd.DataFrame(seg_11_all).loc[0:16], pd.DataFrame(seg_11_all).loc[better],pd.DataFrame(seg_11_all).loc[worse])

F_onewayResult(statistic=array([0.96437774]), pvalue=array([0.38895846]))

In [16]:
#Healthy: Mean static system segregation 
pd.DataFrame(seg_11_all).loc[0:16].mean()

0    0.917022
dtype: float64

In [17]:
#Moderately: Mean static system segregation 
pd.DataFrame(seg_11_all).loc[better].mean()

0    0.878426
dtype: float64

In [18]:
#Severely: Mean static system segregation 
pd.DataFrame(seg_11_all).loc[worse].mean()

0    0.913993
dtype: float64

In [20]:
# loading dynamic functional connectivity matrices
dfnc = np.zeros((n_subjects,n_windows,n_corr))
for n in range(0,n_subjects,1):
    
    dfnc[n] = loadmat(subject_names[n][-40:])['FNCdyn']

In [21]:
# Creating state averages (MEDIAN) per subject
dfnc_1_med = pd.DataFrame(np.zeros((n_subjects, n_corr)))
dfnc_2_med = pd.DataFrame(np.zeros((n_subjects, n_corr)))
dfnc_3_med = pd.DataFrame(np.zeros((n_subjects, n_corr)))

for n in range(0,n_subjects,1):
        dfnc_1_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==1]).median(axis=0))
        #dfnc_1_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==1]).mean(axis=0))
        dfnc_1_med = pd.DataFrame(dfnc_1_med)
        dfnc_2_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==2]).median(axis=0))
        #dfnc_2_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==2]).mean(axis=0))
        dfnc_2_med = pd.DataFrame(dfnc_2_med)
        dfnc_3_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==3]).median(axis=0))
        #dfnc_3_med.loc[n] = np.array(pd.DataFrame(dfnc[n][states.loc[n]==3]).mean(axis=0))
        dfnc_3_med = pd.DataFrame(dfnc_3_med)

In [80]:
# Computing system segregation for median dynamic connectivity matrices
seg_11_all = []
seg_12_all = []
seg_13_all = []

for n in range(48):
    dfnc_1_group_med = pd.DataFrame(return_dfnc_from_vector(dfnc_1_med.loc[n]))
    dfnc_2_group_med = pd.DataFrame(return_dfnc_from_vector(dfnc_2_med.loc[n]))
    dfnc_3_group_med = pd.DataFrame(return_dfnc_from_vector(dfnc_3_med.loc[n]))
    
    dfnc_1_group_med_SM = dfnc_1_group_med.loc[0:7,0:7]*(dfnc_1_group_med.loc[0:7,0:7]>0)
    dfnc_1_group_med_SM = np.array(dfnc_1_group_med_SM)[np.triu_indices(8,1)]
    dfnc_1_group_med_SC = dfnc_1_group_med.loc[8:10,8:10]*(dfnc_1_group_med.loc[8:10,8:10]>0)
    dfnc_1_group_med_SC = np.array(dfnc_1_group_med_SC)[np.triu_indices(3,1)]
    dfnc_1_group_med_CB = dfnc_1_group_med.loc[11:12,11:12]*(dfnc_1_group_med.loc[11:12,11:12]>0)
    dfnc_1_group_med_CB = np.array(dfnc_1_group_med_CB)[np.triu_indices(2,1)]
    within_1 = np.concatenate([dfnc_1_group_med_SM, dfnc_1_group_med_SC, dfnc_1_group_med_CB])
    within_1 = np.sum(within_1)/(len(dfnc_1_group_med_SM) +len(dfnc_1_group_med_SC)+len(dfnc_1_group_med_CB)) 
    dfnc_1_group_med_SM_SC_CB = np.array(dfnc_1_group_med.loc[8:,0:7]*(dfnc_1_group_med.loc[8:,0:7]>0))
    dfnc_1_group_med_SC_CB = np.array(dfnc_1_group_med.loc[8:10,11:]*(dfnc_1_group_med.loc[8:10,11:]>0))
    between_1 = np.concatenate([np.ravel(dfnc_1_group_med_SM_SC_CB),np.ravel(dfnc_1_group_med_SC_CB)])
    between_1 = np.sum(between_1)/(len(np.ravel(dfnc_3_group_med_SM_SC_CB)) +len(np.ravel(dfnc_3_group_med_SC_CB))) 
    seg_11 = (within_1 - between_1) / within_1
    seg_11_all.append(seg_11)
    
    dfnc_2_group_med_SM = dfnc_2_group_med.loc[0:7,0:7]*(dfnc_2_group_med.loc[0:7,0:7]>0)
    dfnc_2_group_med_SM = np.array(dfnc_2_group_med_SM)[np.triu_indices(8,1)]
    dfnc_2_group_med_SC = dfnc_2_group_med.loc[8:10,8:10]*(dfnc_2_group_med.loc[8:10,8:10]>0)
    dfnc_2_group_med_SC = np.array(dfnc_2_group_med_SC)[np.triu_indices(3,1)]
    dfnc_2_group_med_CB = dfnc_2_group_med.loc[11:12,11:12]*(dfnc_2_group_med.loc[11:12,11:12]>0)
    dfnc_2_group_med_CB = np.array(dfnc_2_group_med_CB)[np.triu_indices(2,1)]
    within_2 = np.concatenate([dfnc_2_group_med_SM, dfnc_2_group_med_SC, dfnc_2_group_med_CB])
    within_2 = np.sum(within_2)/(len(dfnc_2_group_med_SM) +len(dfnc_2_group_med_SC)+len(dfnc_2_group_med_CB)) 
    dfnc_2_group_med_SM_SC_CB = np.array(dfnc_2_group_med.loc[8:,0:7]*(dfnc_2_group_med.loc[8:,0:7]>0))
    dfnc_2_group_med_SC_CB = np.array(dfnc_2_group_med.loc[8:10,11:]*(dfnc_2_group_med.loc[8:10,11:]>0))
    between_2 = np.concatenate([np.ravel(dfnc_2_group_med_SM_SC_CB),np.ravel(dfnc_2_group_med_SC_CB)])
    between_2 = np.sum(between_2)/(len(np.ravel(dfnc_3_group_med_SM_SC_CB)) +len(np.ravel(dfnc_3_group_med_SC_CB))) 
    seg_22 = (within_2 - between_2) / within_2
    seg_12_all.append(seg_22)
    
    dfnc_3_group_med_SM = dfnc_3_group_med.loc[0:7,0:7]*(dfnc_3_group_med.loc[0:7,0:7]>0)
    dfnc_3_group_med_SM = np.array(dfnc_3_group_med_SM)[np.triu_indices(8,1)]
    dfnc_3_group_med_SC = dfnc_3_group_med.loc[8:10,8:10]*(dfnc_3_group_med.loc[8:10,8:10]>0)
    dfnc_3_group_med_SC = np.array(dfnc_3_group_med_SC)[np.triu_indices(3,1)]
    dfnc_3_group_med_CB = dfnc_3_group_med.loc[11:12,11:12]*(dfnc_2_group_med.loc[11:12,11:12]>0)
    dfnc_3_group_med_CB = np.array(dfnc_3_group_med_CB)[np.triu_indices(2,1)]
    within_3 = np.concatenate([dfnc_3_group_med_SM, dfnc_3_group_med_SC, dfnc_3_group_med_CB])
    within_3 = np.sum(within_3)/(len(dfnc_3_group_med_SM) +len(dfnc_3_group_med_SC)+len(dfnc_3_group_med_CB)) 
    dfnc_3_group_med_SM_SC_CB = np.array(dfnc_3_group_med.loc[8:,0:7]*(dfnc_3_group_med.loc[8:,0:7]>0))
    dfnc_3_group_med_SC_CB = np.array(dfnc_3_group_med.loc[8:10,11:]*(dfnc_3_group_med.loc[8:10,11:]>0))
    between_3 = np.concatenate([np.ravel(dfnc_3_group_med_SM_SC_CB),np.ravel(dfnc_3_group_med_SC_CB)])
    between_3 = np.sum(between_3)/(len(np.ravel(dfnc_3_group_med_SM_SC_CB)) +len(np.ravel(dfnc_3_group_med_SC_CB))) 
    seg_33 = (within_3 - between_3) / within_3
    seg_13_all.append(seg_33)


In [81]:
# System segregation of State 1
pd.DataFrame(seg_11_all).mean()

0    0.950746
dtype: float64

In [83]:
# System segregation of State 1
pd.DataFrame(seg_12_all).mean()

0    0.885219
dtype: float64

In [85]:
# System segregation of State 1
pd.DataFrame(seg_13_all).mean()

0    0.827221
dtype: float64

In [87]:
# Testing for group differences between the three states:
stats.f_oneway(pd.DataFrame(seg_11_all).dropna(), pd.DataFrame(seg_12_all).dropna(),pd.DataFrame(seg_13_all).dropna())

F_onewayResult(statistic=array([18.80643062]), pvalue=array([8.42876255e-08]))

In [88]:
# Post-hoc t-test: State 2 vs. State 3
stats.ttest_ind(pd.DataFrame(seg_12_all).dropna(),pd.DataFrame(seg_13_all).dropna())

Ttest_indResult(statistic=array([2.77212549]), pvalue=array([0.00692516]))

In [89]:
# Post-hoc t-test: State 2 vs. State 3
stats.ttest_ind(pd.DataFrame(seg_11_all).dropna(),pd.DataFrame(seg_13_all).dropna())

Ttest_indResult(statistic=array([6.42818378]), pvalue=array([1.17401674e-08]))

In [90]:
# Post-hoc t-test: State 1 vs. State 2
stats.ttest_ind(pd.DataFrame(seg_11_all).dropna(),pd.DataFrame(seg_12_all).dropna())

Ttest_indResult(statistic=array([3.53211101]), pvalue=array([0.00069173]))

In [91]:
# FDR-correction for multiple comparisons: All of the post-hoc tests stay significant
statsmodels.stats.multitest.multipletests([0.00692516,1.17401674e-08,0.00069173], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)


(array([ True,  True,  True]),
 array([6.92516000e-03, 3.52205022e-08, 1.03759500e-03]),
 0.016952427508441503,
 0.016666666666666666)

In [24]:
# Statistical comparison of system segregation between subjects (each represented by 159 time windows: therefore 159 windows*48 subjects = 7632 )
a = np.ravel(dfnc)
a = np.reshape(a,(7632,78))

In [25]:
seg_11_all = []

for n in range(7632):
    dfnc_1_group_med = pd.DataFrame(return_dfnc_from_vector(a[n]))

    dfnc_1_group_med_SM = dfnc_1_group_med.loc[0:7,0:7]*(dfnc_1_group_med.loc[0:7,0:7]>0)
    dfnc_1_group_med_SM = np.array(dfnc_1_group_med_SM)[np.triu_indices(8,1)]
    dfnc_1_group_med_SC = dfnc_1_group_med.loc[8:10,8:10]*(dfnc_1_group_med.loc[8:10,8:10]>0)
    dfnc_1_group_med_SC = np.array(dfnc_1_group_med_SC)[np.triu_indices(3,1)]
    dfnc_1_group_med_CB = dfnc_1_group_med.loc[11:12,11:12]*(dfnc_1_group_med.loc[11:12,11:12]>0)
    dfnc_1_group_med_CB = np.array(dfnc_1_group_med_CB)[np.triu_indices(2,1)]
    within_1 = np.concatenate([dfnc_1_group_med_SM, dfnc_1_group_med_SC, dfnc_1_group_med_CB])
    within_1 = np.sum(within_1)/(len(dfnc_1_group_med_SM) +len(dfnc_1_group_med_SC)+len(dfnc_1_group_med_CB)) 
    dfnc_1_group_med_SM_SC_CB = np.array(dfnc_1_group_med.loc[8:,0:7]*(dfnc_1_group_med.loc[8:,0:7]>0))
    dfnc_1_group_med_SC_CB = np.array(dfnc_1_group_med.loc[8:10,11:]*(dfnc_1_group_med.loc[8:10,11:]>0))
    between_1 = np.concatenate([np.ravel(dfnc_1_group_med_SM_SC_CB),np.ravel(dfnc_1_group_med_SC_CB)])
    between_1 = np.sum(between_1)/(len(np.ravel(dfnc_1_group_med_SM_SC_CB)) +len(np.ravel(dfnc_1_group_med_SC_CB))) 
    seg_11 = (within_1 - between_1) / within_1
    seg_11_all.append(seg_11)


In [60]:
# Healthy controls: System segregation
seg_11_control = []
for sub in range(0,17):
    sub_raveled = sub*159
    for sub_rav in range(sub_raveled, (sub_raveled+159)):
        seg_11_control.append(pd.DataFrame(seg_11_all).loc[sub_rav][0])

In [65]:
pd.DataFrame(seg_11_control).mean()

0    0.833723
dtype: float64

In [47]:
# Moderately affected patients: System segregation
seg_11_better = []
for sub in better:
    sub_raveled = sub*159
    for sub_rav in range(sub_raveled, (sub_raveled+159)):
        seg_11_better.append(pd.DataFrame(seg_11_all).loc[sub_rav][0])

In [50]:
pd.DataFrame(seg_11_better).mean()

0    0.809731
dtype: float64

In [51]:
# Severely affected patients: System segregation
seg_11_worse = []
for sub in worse:
    sub_raveled = sub*159
    for sub_rav in range(sub_raveled, (sub_raveled+159)):
        seg_11_worse.append(pd.DataFrame(seg_11_all).loc[sub_rav][0])

In [52]:
pd.DataFrame(seg_11_worse).mean()

0    0.856089
dtype: float64

In [61]:
# One-way ANOVA for overall group-effect
stats.f_oneway(np.array(seg_11_control), np.array(seg_11_better), np.array(seg_11_worse))

F_onewayResult(statistic=61.89625606353308, pvalue=2.1605358218569088e-27)

In [63]:
# Post-hoc t-test: Controls vs moderately affected patients 
stats.ttest_ind(np.array(seg_11_control),np.array(seg_11_better))

Ttest_indResult(statistic=6.372654803307531, pvalue=2.0072583130281055e-10)

In [64]:
# Post-hoc t-test: Controls vs severely affected patients 
stats.ttest_ind(np.array(seg_11_control),np.array(seg_11_worse))

Ttest_indResult(statistic=-5.342893835621885, pvalue=9.573273286120495e-08)

In [66]:
# Post-hoc t-test: Moderately vs severely affected patients 
stats.ttest_ind(np.array(seg_11_better),np.array(seg_11_worse))

Ttest_indResult(statistic=-10.51713165975335, pvalue=1.3426766362264276e-25)

In [92]:
# FDR-correction for multiple comparisons: All of the post-hoc tests stay significant
statsmodels.stats.multitest.multipletests([2.0072583130281055e-10,9.573273286120495e-08,1.3426766362264276e-25], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)


(array([ True,  True,  True]),
 array([3.01088747e-10, 9.57327329e-08, 4.02802991e-25]),
 0.016952427508441503,
 0.016666666666666666)